In [1]:
import pandas as pd
import numpy as np
import json

## adding z coords to geojson pts

In [2]:
#with open('../gis/gurla_z_pts.geojson') as f:
#    gp = json.load(f)

In [3]:
#for feat in gp['features']:
#    coords = feat['geometry']['coordinates']
#    z_val = feat['properties']['Z']
#    coords.append(z_val)

In [4]:
#with open('../gis/gurla_z_pts.geojson', 'w') as f:
#    json.dump(gp, f)

In [6]:
gp['features'][0]

{'geometry': {'coordinates': [-48162.7720081706,
   3350150.7917627767,
   3685,
   3685,
   3685],
  'type': 'Point'},
 'properties': {'X': -48162.772008, 'Y': 3350150.791763, 'Z': 3685, 'id': 0},
 'type': 'Feature'}

## adding z coords to geojson polys

### make pt table

In [7]:
pdf = pd.read_csv('../gis/gurla_z_pts.csv', index_col=3)

pdf.head()

,X,Y,Z
id,,,
0,-48162.772008,3350150.791763,3685
1,-50157.176651,3351825.986467,3681
2,-51194.059027,3354386.865733,3841
3,-53080.208694,3356174.076748,3948
4,-54299.894490,3358102.938650,4118


### get geojson tris

In [56]:
with open('../gis/gurla_tris.geojson') as f:
    gt = json.load(f)

### get z values

In [57]:
gt['features'][0]['geometry']['coordinates']#[0][0]

[[[-55783.583006836, 3342716.71580536],
  [-54677.8965490291, 3339938.7432105276],
  [-56725.8969340146, 3341331.705430983],
  [-55783.583006836, 3342716.71580536]]]

In [58]:
gc = gt.copy()

In [59]:
for feat in gc['features']:
    coors = feat['geometry']['coordinates'][0]
    zs = pdf.loc[(int(feat['properties']['POINTA']), int(feat['properties']['POINTB']), 
                  int(feat['properties']['POINTC']), int(feat['properties']['POINTA'])), 'Z'
                 ].values
    for i, coor in enumerate(coors):
        coor.append(float(zs[i]))

In [60]:
gc['features'][-1]

{'geometry': {'coordinates': [[[-57886.2658870933,
     3407087.5218043914,
     -20455.0],
    [-54899.2065167453, 3407362.546040953, -20455.0],
    [-55140.9567051379, 3405082.4065878605, -15455.0],
    [-57886.2658870933, 3407087.5218043914, -20455.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 175.0, 'POINTB': 174.0, 'POINTC': 142.0},
 'type': 'Feature'}

### get centers and area

$S = \frac{|AB \times AC|}{2}$

In [61]:
def area(coords):
    ab = np.array(coords[0]) - np.array(coords[1])
    ac = np.array(coords[0]) - np.array(coords[2])
    
    return np.linalg.norm(np.cross(ab, ac)) / 2e6

In [62]:
def center(coords):
    return ((np.array(coords[0]) + np.array(coords[1]) 
             + np.array(coords[2])) / 3).tolist()

center(coors)

[-55975.476369658834, 3406510.824811069, -18788.333333333332]

In [63]:
for feat in gc['features']:
    coords = feat['geometry']['coordinates'][0]
    feat['properties']['area_sq_km'] = area(coords)
    
    feat['properties']['center'] = center(coords)

In [64]:
gc['features'][100]

{'geometry': {'coordinates': [[[-63078.7116583341, 3368647.63774455, -816.0],
    [-56488.0645559977, 3367841.8357556644, 4045.0],
    [-56808.9791004168, 3366516.191889601, 4263.0],
    [-63078.7116583341, 3368647.63774455, -816.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 36.0,
  'POINTB': 9.0,
  'POINTC': 8.0,
  'area_sq_km': 5.6830856866119914,
  'center': [-58791.91843824953, 3367668.5551299383, 2497.3333333333335]},
 'type': 'Feature'}

### get strike and dip

In [65]:
import halfspace.projections as hsp

In [66]:
for feat in gc['features']:
    coords = feat['geometry']['coordinates'][0]
    
    coo = [[co[0], co[1], -co[2]] for co in coords[:-1]]
    
    s, d = hsp.strike_dip_from_3_xyz(coo[0], coo[1], coo[2])
    
    feat['properties']['strike'], feat['properties']['dip'] = s, d

In [67]:
hsp.strike_dip_from_3_xyz(*[[co[0], co[1], -co[2]] for co in coors[:-1]])

(185.26049999942541, 65.787778323950604)

In [68]:
gc['features'][100]

{'geometry': {'coordinates': [[[-63078.7116583341, 3368647.63774455, -816.0],
    [-56488.0645559977, 3367841.8357556644, 4045.0],
    [-56808.9791004168, 3366516.191889601, 4263.0],
    [-63078.7116583341, 3368647.63774455, -816.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 36.0,
  'POINTB': 9.0,
  'POINTC': 8.0,
  'area_sq_km': 5.6830856866119914,
  'center': [-58791.91843824953, 3367668.5551299383, 2497.3333333333335],
  'dip': 37.68134833327008,
  'strike': 244.44859892337777},
 'type': 'Feature'}

### get rake

In [69]:
def plunge_from_sd_trend(strike, dip, trend, angle='degrees', out_angle='degrees'):
    ''' Calculates the plunge of a trend (azimuth) when projected onto a dipping plane.
    This is the calculation for apparent dip, but may also be other sorts of projections,
    such as the plunge of a lineation of known azimuth on a plane of known attitude.
    
    Returns a float scalar or vector of plunges.
    
    Inputs:
    strike: Strike of plane
    dip: Dip of plane
    trend: Azimuth of line to be projected onto the plane
    angle: Unit of strike, dip, trend (degrees (default) or radians.)
    out_angle: Unit of plunge (degrees (default) or radians.)
    
    All inputs should be in right-hand rule.
    '''
    if angle == 'degrees':
        s = np.radians(strike)
        d = np.radians(dip)
        t = np.radians(trend)
    else:
        s = strike
        d = dip
        t = trend
        
    dip_trend_angle = s + np.pi/2 - t
    
    plunge = np.arctan( np.cos(dip_trend_angle) * np.tan(d))
    
    if out_angle == 'degrees':
        plunge = np.degrees(plunge)
    
    return plunge


def rake_from_sd_trend(strike, dip, trend, angle='degrees',
                       out_angle='degrees', aki_richards=False):
    '''
    Calculates the rake'''
    

    plunge = plunge_from_sd_trend(strike, dip, trend, angle=angle,
                                 out_angle=out_angle)
    rake = rake_from_dip_plunge(dip, plunge, angle=angle,
                                out_angle=out_angle)
    
    # some more stuff to get rake into Aki and Richards convention
    # (0 = rl, -90 = normal, +/- 180 = ll, 90 = reverse)
    if aki_richards == True:
        rake = rake_to_aki_richards(strike, dip, trend, rake,
                                   angle=angle, out_angle=out_angle)
        
    return rake


def rake_from_dip_plunge(dip, plunge, angle='degrees',
                         out_angle='degrees'):
    if angle == 'degrees':
        d = np.radians(dip)
        p = np.radians(plunge)
    elif angle == 'radians':
        d = dip
        p = plunge
    
    rake = np.arcsin( np.sin(p) / np.sin(d))
    if out_angle == 'degrees':
        rake = np.degrees(rake)
    elif out_angle == 'radians':
        pass
    else:
        raise Exception('out_angle needs to be in "degrees" or "radians"')
    return rake


def rake_to_aki_richards(strike, dip, trend, rake, 
                         angle='degrees', out_angle='degrees'):
    '''
    Takes a regular rake angle, as well as the strike, dip and azimuthal, directional
    trend of slip and returns the rake in Aki and Richards convention.'''
    
    rake *= -1
    if angle == 'radians':
        strike = np.degrees(strike)
        trend = np.degrees(trend)
        rake = np.degrees(rake)
        
    if np.isscalar(dip): 
        trend_strike_diff = angle_difference(strike, trend)
    else:   # work w/ dip ranges and scalar strike, trends
        trend_strike_diff = np.ones(len(dip)) * angle_difference(strike, trend)
    
    if np.isscalar(rake):
        if 90 < trend_strike_diff <= 180:
            rake = -(180 + rake)
        elif -90 >= trend_strike_diff >= -180:
            rake = 180 - rake
    else:
        rake[ (90 < trend_strike_diff) & (trend_strike_diff < 180) ] += 180
        rake[ (90 < trend_strike_diff) & (trend_strike_diff < 180) ] *= -1
        rake[(-90 > trend_strike_diff) & (trend_strike_diff > -180)] *= -1
        rake[(-90 > trend_strike_diff) & (trend_strike_diff > -180)] += 180
        
    if out_angle == 'radians':
        rake = np.radians(rake)
    return rake

In [70]:
def unwrap(azimuth):
    if np.isscalar(azimuth):
        while azimuth < 0:
            azimuth += 360
        while azimuth > 360:
            azumuth -= 360
    else:
        azimuth = np.array(azimuth)
        while np.any([azimuth < 0]):
            azimuth[azimuth < 0] += 360
        
        while np.any([azimuth > 360]):
            azimuth[azimuth > 360] -= 360
    return azimuth
    

In [71]:
def angle_difference(trend1, trend2, return_abs=False, angle='degrees'):
    '''
    Calculates the angular difference between two trends.
    'return_abs' (boolean) is whether the absolute value of the
    difference is to be returned, default False.
    'angle' specifies whether the units are in 'degrees' (default)
    or 'radians'.
    '''
    if np.isscalar(trend1) and not np.isscalar(trend2):
        trend1 = np.ones(len(trend2)) * trend1
    elif np.isscalar(trend2) and not np.isscalar(trend2):
        trend2 = np.ones(len(trend1)) * trend2

    if angle == 'radians':
        trend1 = np.degrees(trend1)
        trend2 = np.degrees(trend2)

    if np.isscalar(trend1) and np.isscalar(trend2):
        diff = angle_difference_scalar(trend1, trend2)
    else:
        diff = angle_difference_vector(trend1, trend2)

    if angle == 'radians':
        diff = np.radians(diff)

    return diff if return_abs == False else np.abs(diff)


def angle_difference_scalar(trend1, trend2):
    '''
    Angular difference between two trends, in degrees.
    '''
    difference = trend2 - trend1
    while difference < -180:
        difference += 360
    while difference > 180:
        difference -= 360
    return difference


def angle_difference_vector(trend1_vec, trend2_vec):
    '''Angular difference between to vectors of trends, in degrees.'''
    trend1_vec = np.array(trend1_vec)
    trend2_vec = np.array(trend2_vec)
    difference = trend2_vec - trend1_vec
    difference[difference < -180] += 360
    difference[difference > 180] -= 360
    
    return difference

In [72]:
gc['features'][100]

{'geometry': {'coordinates': [[[-63078.7116583341, 3368647.63774455, -816.0],
    [-56488.0645559977, 3367841.8357556644, 4045.0],
    [-56808.9791004168, 3366516.191889601, 4263.0],
    [-63078.7116583341, 3368647.63774455, -816.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 36.0,
  'POINTB': 9.0,
  'POINTC': 8.0,
  'area_sq_km': 5.6830856866119914,
  'center': [-58791.91843824953, 3367668.5551299383, 2497.3333333333335],
  'dip': 37.68134833327008,
  'strike': 244.44859892337777},
 'type': 'Feature'}

In [73]:
for feat in gc['features']:
    prop = feat['properties']
    prop['rake'] = rake_from_sd_trend(prop['strike'], prop['dip'], 277,
                                      aki_richards=True)
    prop['rake_err'] = 10

In [74]:
gc['features'][100]

{'geometry': {'coordinates': [[[-63078.7116583341, 3368647.63774455, -816.0],
    [-56488.0645559977, 3367841.8357556644, 4045.0],
    [-56808.9791004168, 3366516.191889601, 4263.0],
    [-63078.7116583341, 3368647.63774455, -816.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 36.0,
  'POINTB': 9.0,
  'POINTC': 8.0,
  'area_sq_km': 5.6830856866119914,
  'center': [-58791.91843824953, 3367668.5551299383, 2497.3333333333335],
  'dip': 37.68134833327008,
  'rake': -38.888365294432063,
  'rake_err': 10,
  'strike': 244.44859892337777},
 'type': 'Feature'}

### average stresses at points and center to get values per tri
- get points df (only coords)
- get tensorial stresses at points
- average
- make new fault_df w/ averages and geom for each tri
- do regular inversion from there

In [75]:
import sys
from importlib import reload
sys.path.append('../../scripts/')
import utils


In [76]:
reload(utils)

<module 'utils' from '../../scripts/utils.py'>

x        y         z
tri vertex                             
0   a      -55783.58  3342717    -14985
    b       -54677.9  3339939    -19768
    c       -56725.9  3341332    -19747
    m      -55729.13  3341329 -18166.67

In [77]:
gurla_pts_df = utils.tri_dict_to_df(gc, fault_name='gurla')

In [78]:
gurla_pts_df.to_csv('../data/fault_data/gurla_pts_df.csv')

In [79]:
gc['features'][0]

{'geometry': {'coordinates': [[[-55783.583006836, 3342716.71580536, -14985.0],
    [-54677.8965490291, 3339938.7432105276, -19768.0],
    [-56725.8969340146, 3341331.705430983, -19747.0],
    [-55783.583006836, 3342716.71580536, -14985.0]]],
  'type': 'Polygon'},
 'properties': {'POINTA': 130.0,
  'POINTB': 204.0,
  'POINTC': 203.0,
  'area_sq_km': 6.2515942816401573,
  'center': [-55729.12549662657, 3341329.0548156234, -18166.666666666668],
  'dip': 70.619256263885447,
  'rake': 73.88142642292091,
  'rake_err': 10,
  'strike': 325.94902891533388},
 'type': 'Feature'}

In [80]:
with open('../data/fault_data/gurla_tris.geojson', 'w') as f:
    json.dump(gc, f)